In [1]:
''' IMPORT ALL THE THINGS '''
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, smart_resize
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

In [2]:
''' DEFINE VARIABLES '''

CWD = os.getcwd().replace('\\','/') # get current workspace directory
IMAGE_DIR = f'{CWD}/images'
TRAIN_DATASET_DIR = f'{IMAGE_DIR}/train'
TEST_DATASET_DIR = f'{IMAGE_DIR}/test'
VAL_DATASET_DIR = f'{IMAGE_DIR}/validation'

CATEGORIES = ['clementine','grapefruit','orange']

IMAGE_RESIZE = (224, 224) # size of image to put in Model

AUGMENT_N_IMAGE = 5 # how many time each image get the augmentation

AUGMENT_SAVE_PREFIX = 'aug'

Detail_model = 'resnet50_fc512_do01_fc256_fc3_aug'


In [3]:
''' Create Data Augmentation '''
def run_augmentation( isRun = 0 ):

    if isRun:

        print('Running Augmentation...\n')

        datagen = ImageDataGenerator(
            rotation_range=45,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='reflect' # optional : nearest, constant(125), reflect, wrap
        ) 

        try:
            for category in CATEGORIES:

                train_image_in_directory = [] 

                train_category_directory = f"{TRAIN_DATASET_DIR}/{category}"

                for i, image_name in enumerate(os.listdir(train_category_directory)):
                    
                    file_path = f"{train_category_directory}/{image_name}"
                    
                    if (image_name.find(AUGMENT_SAVE_PREFIX) != -1):  # delete old augmentation
                        if (os.path.exists(file_path)):
                            os.remove(file_path)

                    elif (image_name.split('.')[1] in ['png', 'jpg', 'jpeg']):

                        image = load_img(file_path)
                        image = smart_resize(image, IMAGE_RESIZE)
                        image = img_to_array(image)
                        train_image_in_directory.append(image)

                train_image_in_directory = np.array(train_image_in_directory)
                print(f"{category} : ", end="")
                print(train_image_in_directory.shape)

                i = 0
                
                # save_to_dir (option) : f'{train_category_directory}', 'augmented'
                for batch in datagen.flow(train_image_in_directory, batch_size=1, save_to_dir=f'{train_category_directory}', save_prefix=AUGMENT_SAVE_PREFIX, save_format='jpg'):
                    i += 1
                    # datagen.flow make infinite loop NEED A MANUAL BREAK!!!
                    if i >= len(train_image_in_directory) * AUGMENT_N_IMAGE:
                        break

            print('\nAugmentation Success!!!')

        except Exception as e:

            print('\nAugmentation Failed...')
            print('Exception is\n',e)
    
    


In [ ]:
run_augmentation(1)

Running Augmentation...

clementine : (485, 224, 224, 3)


In [ ]:
''' LOAD DATA into Variables '''
try:
    print("Loading data...\n")
    
    print('-----------Train-----------')
    #Load train data

    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_DATASET_DIR, # train_data path
        image_size=IMAGE_RESIZE, # each image resize
        batch_size=1,
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(train_dataset.class_names)  # class check
    print(train_dataset)


    print("-----------Validation-----------")
    #Load validation data
    valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        VAL_DATASET_DIR,  # validation_data path
        image_size=IMAGE_RESIZE,  # each image resize
        batch_size=1, 
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

    print(valid_dataset.class_names)  # class check
    print(valid_dataset)

    print("\nLoad data Success!!!")
    
except Exception as e:

    print("\nLoad data failed...")
    print("Exception is",e)




In [ ]:
''' Get pre train model '''
resnet = ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
resnet.trainable = False

In [ ]:
''' Training and Display result'''

# create model architecture
model = tf.keras.models.Sequential([
                                resnet,
                                tf.keras.layers.Flatten(),
                                tf.keras.layers.Dense(512, activation='relu'),
                                tf.keras.layers.Dropout(0.1),
                                tf.keras.layers.Dense(256, activation='relu'),
                                tf.keras.layers.Dense(3, activation='softmax')
])

# compile
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
los = tf.keras.losses.categorical_crossentropy
model.compile( optimizer = opt , loss = los , metrics = ['accuracy'] )

# Training
model.fit(
    
    train_dataset,  # both data and label cause we use preprocessing
    validation_data=valid_dataset,
    batch_size=256,
    epochs=10, ### can be change
    shuffle=True 
)

# Plot the result
fig, axs = plt.subplots(1,2, figsize=(16,5))
fig.suptitle(f"{Detail_model}")
axs[0].plot(model.history.history['accuracy'])
axs[0].plot(model.history.history['val_accuracy'])
axs[0].set_title('Accuracy')
axs[0].legend(['train', 'validation'], loc='upper left')
axs[0].set(xlabel='epoch', ylabel='accuracy')

axs[1].plot(model.history.history['loss'])
axs[1].plot(model.history.history['val_loss'])
axs[1].set_title('Loss')
axs[1].legend(['train', 'validation'], loc='upper left')
axs[1].set(xlabel='epoch', ylabel='loss')

''' Save the model '''
model.save(f'{CWD}/Model/{Detail_model}')

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        TEST_DATASET_DIR,  # test_dataset path
        image_size=IMAGE_RESIZE,  # each image resize
        batch_size=1, 
        label_mode='categorical',
        color_mode='rgb',
        shuffle=True
    )

print(test_dataset.class_names)  # class check
print(test_dataset)

print("\nLoad data Success!!!")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.summary()

In [ ]:
# dir_path = f'{CWD}/images/validation/'
# for j in os.listdir(dir_path):
#     for i in os.listdir(dir_path + '/' + j):
#         img = tf.keras.preprocessing.image.load_img(dir_path + '/' +j+'/'+i, target_size=(224,224))
#         plt.imshow(img)
#         plt.show()

#         X= tf.keras.preprocessing.image.img_to_array(img)
#         X= np.expand_dims(X,axis=0)
#         image = np.vstack([X])
#         val = model.predict(image)
#         '''print(val)
#         print(val[0][0])
#         print(val[0][1])
#         print(val[0][2])
#         print(np.argmax(val)) '''

#         if val[0][0] > val[0][1] and val[0][0] > val[0][2] :
#             print('this is Clementine : '+i)
#             if j == 'clementine':
#                 print('Correct')
#             else:
#                 print('incorrect')
            


#         elif val[0][1] > val[0][0] and val[0][1] > val[0][2] :
#             print('this is Grapefruit : '+i)
#             if j == 'grapefruit':
#                 print('Correct')
#             else:
#                 print('incorrect')
           

#         elif val[0][2] > val[0][0] and val[0][2] > val[0][1] :
#             print('this is Orange : '+i)
#             if j == 'orange':
#                 print('Correct')
#             else:
#                 print('incorrect')
            


In [ ]:
predict_model = load_model(f'{CWD}/Model/{Detail_model}')

In [ ]:
img = tf.keras.preprocessing.image.load_img(f'{CWD}/images/test/clementine/1000.jpg', target_size=(224,224))
print(img)
plt.imshow(img)
plt.show()

X= tf.keras.preprocessing.image.img_to_array(img)
X= np.expand_dims(X,axis=0)
image = np.vstack([X])
val = predict_model.predict(image)
print("Val : ",end="")
print(type(val[0][0]))